In [10]:
import pandas as pd
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
from pandas import json_normalize
import ast
nltk.download('vader_lexicon')
import pyarrow

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


User Reviews

In [11]:
#Ruta del archivo JSON
data_list = []
file_path = 'australian_user_reviews.json'

#Abrir el archivo y procesar cada línea
with open(file_path, 'r', encoding='utf-8') as file:
    for line in file:
        try:
            # Usar ast.literal_eval para convertir la línea en un diccionario
            json_data = ast.literal_eval(line)
            data_list.append(json_data)
        except ValueError as e:
            print(f"Error en la línea: {line}")
            continue

#Crear un DataFrame a partir de la lista de diccionarios
userReviewsDf = pd.DataFrame(data_list)

In [12]:
userReviewsDf.to_parquet('userReviews.parquet',index=False)

In [13]:
userReviewsExplodedDf=userReviewsDf.explode('reviews').reset_index(drop=True)

In [14]:
userReviewsExplodedDf['review']=userReviewsExplodedDf['reviews'].apply(lambda x: x.get('review') if isinstance(x, dict) and 'review' in x else None)
userReviewsExplodedDf['recommend']=userReviewsExplodedDf['reviews'].apply(lambda x: x.get('recommend') if isinstance(x, dict) and 'recommend' in x else None)
userReviewsExplodedDf['item_id']=userReviewsExplodedDf['reviews'].apply(lambda x: x.get('item_id') if isinstance(x, dict) and 'item_id' in x else None)
userReviewsExplodedDf['review_year']=userReviewsExplodedDf['reviews'].apply(lambda x: x.get('posted') if isinstance(x, dict) and 'posted' in x else None)


In [15]:
userReviewsExplodedDf['review_year'] = userReviewsExplodedDf['review_year'].str.replace('Posted ', '', regex=False)
userReviewsExplodedDf['review_year']=pd.to_datetime(userReviewsExplodedDf['review_year'],format='mixed',errors='coerce')
userReviewsExplodedDf['review_year']=userReviewsExplodedDf['review_year'].dt.year

In [16]:
def analyze_sentiment(text):
    if text is None:
        return '1'

    sia = SentimentIntensityAnalyzer()
    sentiment_score = sia.polarity_scores(text)['compound']

    if text:
        if sentiment_score >= 0.05:
            return '2'
        elif sentiment_score <= -0.05:
            return '0'
        else:
            return '1'

In [17]:
userReviewsExplodedDf['sentiment_analysis']=userReviewsExplodedDf['review'].apply(analyze_sentiment)

In [18]:
del userReviewsExplodedDf['reviews']
del userReviewsExplodedDf['review']
del userReviewsExplodedDf['user_url']

In [22]:
userReviewsExplodedDf.to_parquet('userReviewsExploded.parquet',index=False)

In [24]:
userReviewsExplodedData=pd.read_parquet('userReviewsExploded.parquet')